In [1]:
from utils import calculate_auc
from data_processor import TennisProcessor
from train_test_split_games import TennisSplitter
from true_skill_through_time import TrueSkillThroughTimeApplied
import json

%load_ext autoreload
%autoreload 2

# Run this to generate and save the data

In [2]:
# tennis_instance = TennisProcessor("data/tennis_history.csv")
# tennis_data_df = tennis_instance.process_games_data()
# tennis_data_df.to_parquet("data/tennis_matches_refined_tstt.parquet")

# Split the data into train and test sets

In [3]:
tennis_splitter = TennisSplitter("data/tennis_matches_refined_tstt.parquet", "data/tennis_players_ge_40_matches_lst.json")
tennis_train, tennis_test = tennis_splitter.train_test_split()

100%|██████████████████████████████████████████████████████████████████████████████| 2502/2502 [01:15<00:00, 33.10it/s]


# Tune and Apply the TrueSkill Through Time

In [4]:
trueskill = TrueSkillThroughTimeApplied(tennis_train)

## Only run the below if you'd like to re-tune the hyperparameters

In [5]:
# trueskill.learn_optimal_parameters(game_type='tennis')

## Set the hyperparameters

In [6]:
# MLE: 177618.9780
optimal_gamma = 0.0500
optimal_sigma = 0.5000
optimal_beta = 0.5000
trueskill.set_optimal_parameters(gamma = optimal_gamma, sigma = optimal_sigma, beta = optimal_beta)

In [7]:
skill_curves = trueskill.set_skill_curves()

In [8]:
with open("data/tennis_player_id_inv_map.json") as f:
    tennis_player_id_inv_map = json.load(f)

In [24]:
top_players_lst = [
    "novak-djokovic",
    "roger-federer",
    "pete-sampras",
    "ivan-lendl",
    #"jimmy-connors",
    "rafael-nadal",
    #"john-mcenroe",
    #"bjorn-borg",
    "andre-agassi",
    "lleyton-hewitt",
    # "stefan-edberg",
    # "jim-courier",
    # "gustavo-kuerten",
    # "andy-murray",
    #"ilie-nastase",
    "mats-wilander"
]

In [25]:
top_players_id_lst = [
    tennis_player_id_inv_map[player] for player in top_players_lst
]

In [26]:
top_players_map = {
   'd643': "novak-djokovic",
   'f324': "roger-federer",
   's402': "pete-sampras",
   'l018': "ivan-lendl",
   'c044': "jimmy-connors",
   'n409': "rafael-nadal",
   'm047': "john-mcenroe",
   'b058': "bjorn-borg",
   'a092': "andre-agassi",
   'h432': "lleyton-hewitt",
   'e004': "stefan-edberg",
   'c243': "jim-courier",
   'k293': "gustavo-kuerten",
   'mc10': "andy-murray",
   'n008': "ilie-nastase",
   'w023': "mats-wilander"
}

In [37]:
trueskill.plot_player_skills(players = top_players_id_lst, players_id_mapping=top_players_map, width=1000, height=500, burnin=0, x_axis_format="%Y", y_lims=[-0.6, 2.7], game='tennis')

alt.LayerChart(...)

In [32]:
trueskill.plot_calibration()

C:\Users\Duane\Dropbox\consulting\website\public_repo\true_theta\posts\trueskill\true_skill_through_time.py:251: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_means = df.groupby("win_prob_bucket").agg(


alt.LayerChart(...)

In [35]:
trueskill.plot_calibration_oos(oos_data=tennis_test, game='Tennis')

C:\Users\Duane\Dropbox\consulting\website\public_repo\true_theta\posts\trueskill\true_skill_through_time.py:330: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  x=alt.X("win_prob_midpoint", title="Predicted Win Probability"),


alt.LayerChart(...)

In [20]:
calculate_auc(tennis_test, skill_curves, optimal_beta)

np.float64(0.7067910709395497)